In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 29.5 MB/s 
     |████████████████████████████████| 895 kB 69.0 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 6.5 MB 66.2 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
import random
import scipy.sparse as sp
import torch
import os
import time
import pandas as pd
import pickle
import torch.nn.functional as F
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from transformers import GPT2Tokenizer, GPT2Model
import nltk
from keras_preprocessing.text import Tokenizer
from torch import nn
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve 
from sklearn.metrics import roc_auc_score
from scipy.sparse import coo_matrix, vstack

In [3]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'  # TF 2.1
random.seed(SEED)
#seed(SEED)
np.random.seed(SEED)

In [5]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# BOW

In [20]:
def sentence_process(sentence):
  comment = sentence.replace('\n', ' ')
  while '\"' in comment :
    comment = comment.replace('\"', '')
  while "\'" in comment :
    comment = comment.replace("\'", '')
  while ':' in comment :
    comment = comment.replace(':', '')
  while '.' in comment :
    comment = comment.replace('.', '')
  while '@' in comment :
    comment = comment.replace('@', '')
  while '+' in comment :
    comment = comment.replace('+', '')
  while '=' in comment:
    comment = comment.replace('=', '')
  while '&' in comment :
    comment = comment.replace('&', '')
  while ')' in comment or '(' in comment:
    comment = comment.replace(')', '').replace('(', '')
  while ',' in comment or ':' in comment or ';' in comment:
    comment = comment.replace(":", '').replace(',', '').replace(';', '')
  for c in comment :
    if c.isascii() == False :
      comment = comment.replace(c, '')
  n1 = comment.count('!')
  n2 = comment.count('?')
  for i in range(n1):
    comment = comment.replace('!', ' !')
  for i in range(n2):
    comment = comment.replace('?', ' ?')
  while '  ' in comment :
    comment = comment.replace('  ', ' ')
  return comment

def bow(sentence,dic):
  v=np.zeros(len(dic))
  for word in dic :
    words = sentence.split(' ')
    for w in words :
      if word == w:
        v[dic[word]]+=1
  return(coo_matrix(v))

class BOWDataset :
    def __init__(self, filepath, n_words):
        self.filepath = filepath
        self.n_words = n_words

    def process(self):
        tic = time.time()

        train_df = pd.read_csv(self.filepath + 'train.csv')
        tic = time.time()

        #delete comments that belongs to several classes
        idx_to_del = []
        for row in train_df.itertuples():
          if sum(row[3::])> 1 :
            idx_to_del.append(row[0])
          
        train_df = train_df.drop(idx_to_del)

        cpt = 0
        n_too_long = 0
        toxic = train_df[train_df['toxic'] == 0]
        obscene = toxic[toxic['obscene']==0]
        threat = obscene[obscene['threat']==0]
        insult = threat[threat['insult']==0]
        neutral = insult[insult['identity_hate']==0] #df with neutral comment
        toxic = train_df[train_df['toxic'] == 1]
        obscene = train_df[train_df['obscene'] == 1]
        threat = train_df[train_df['threat'] == 1]
        insult = train_df[train_df['insult'] == 1]
        identity_hate = train_df[train_df['identity_hate'] == 1]
        final_df = pd.concat([toxic, obscene, threat, insult, identity_hate, neutral]) #merge all the df together in the right order
        
        dic = {}
        n_dic=0
        if True :
          for row in final_df.itertuples():
            
            cpt = cpt + 1
            if cpt in [1344, 1345, 1445, 1446, 1528, 1529, 3243, 3244, 3773, 3776, 11888] :
                continue

            if n_dic>=10000:
                break
            print("cpt dic:", cpt)
            
            comment = sentence_process(row[2])
            words = comment.split(' ')
            while '' in words :
              words.remove('')
            for w in words:
              if w.isnumeric() and len(w)>=5:
                words.remove(w)
            words = list(filter(lambda s: 'http' not in s, words))
            comment = ' '.join(words)

            if len(words)>100 : 
              print("Too long")
              n_too_long += 1
              continue

            n_dic=n_dic+1
            for word in words:
              if word not in dic:
                dic[word]=len(dic)

        with open(self.filepath +'dic.pickle', 'wb') as handle:
          pickle.dump(dic, handle)

        #with open(self.filepath +'dic.pickle', 'rb') as handle:
          #dic = pickle.load(handle)

        cpt = 0
        X = []
        self.labels = []
        for row in final_df.itertuples():
      
          cpt = cpt + 1
          print("cpt X", cpt)

          if cpt in [1344, 1345, 1445, 1446, 1528, 1529, 3243, 3244, 3773, 3776, 11888] :
            continue

          if np.shape(X)[0] == self.n_words and np.shape(self.labels[:self.n_words])[0] == self.n_words:
            break
          
          comment = sentence_process(row[2])
          words = comment.split(' ')
          while '' in words :
            words.remove('')
          for w in words:
            if w.isnumeric() and len(w)>=5:
              words.remove(w)
          words = list(filter(lambda s: 'http' not in s, words))
          comment = ' '.join(words)

          if len(words)>100 : 
            print("Too long")
            n_too_long += 1
            continue

          if row[3] == 0 and row[4] == 0 and row[5] ==0 and row[6] ==0 and row[7] == 0 and row[8] == 0:
            self.labels.append(0)
          elif row[3] == 1 :
            self.labels.append(1)
          elif row[4] == 1 :
            self.labels.append(6)
          elif row[5] == 1 :
            self.labels.append(2)
          elif row[6] == 1 :
            self.labels.append(3)
          elif row[7] == 1 :
            self.labels.append(4)
          else: 
            self.labels.append(5)

          X.append(bow(comment,dic))

        X = vstack(X)
        labels = np.array(self.labels)
        print("labels", np.unique(labels))
        with open('/content/drive/My Drive/NLP/X_10000.pickle', 'wb') as handle:
          pickle.dump(X, handle)

        with open('/content/drive/My Drive/NLP/labels_10000.pickle', 'wb') as handle: 
          pickle.dump(labels, handle)
        return X, labels

X, labels = BOWDataset(filepath = '/content/drive/My Drive/NLP/', n_words = 10000).process()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
cpt X 7757
cpt X 7758
cpt X 7759
cpt X 7760
cpt X 7761
Too long
cpt X 7762
cpt X 7763
cpt X 7764
cpt X 7765
Too long
cpt X 7766
cpt X 7767
cpt X 7768
cpt X 7769
cpt X 7770
cpt X 7771
cpt X 7772
cpt X 7773
Too long
cpt X 7774
cpt X 7775
cpt X 7776
cpt X 7777
cpt X 7778
cpt X 7779
cpt X 7780
cpt X 7781
cpt X 7782
Too long
cpt X 7783
cpt X 7784
Too long
cpt X 7785
Too long
cpt X 7786
cpt X 7787
cpt X 7788
cpt X 7789
Too long
cpt X 7790
cpt X 7791
cpt X 7792
cpt X 7793
cpt X 7794
cpt X 7795
cpt X 7796
cpt X 7797
cpt X 7798
cpt X 7799
cpt X 7800
cpt X 7801
Too long
cpt X 7802
cpt X 7803
cpt X 7804
cpt X 7805
cpt X 7806
cpt X 7807
cpt X 7808
cpt X 7809
cpt X 7810
cpt X 7811
Too long
cpt X 7812
cpt X 7813
Too long
cpt X 7814
cpt X 7815
cpt X 7816
Too long
cpt X 7817
cpt X 7818
cpt X 7819
cpt X 7820
Too long
cpt X 7821
Too long
cpt X 7822
cpt X 7823
cpt X 7824
Too long
cpt X 7825
cpt X 7826
cpt X 7827
cpt X 7828
Too 

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

random_states = [0,1,2]

accuracy = []
F1_Score=[]
AUC=[]

tic = time.time()

#class_weights = compute_class_weight(class_weight='balanced',classes=np.unique(labels),y=labels)
#print(class_weights)

for r in random_states :

  print("Random state : ", r, "\n")
  X, labels = shuffle(X, labels, random_state=r)
  num_examples = np.shape(X)[0]
  num_train = int(0.80 * num_examples)
  num_val = int((num_examples-num_train)/2)
  X_train, y_train = X[:num_train], labels[:num_train]
  X_test, y_test = X[(num_train+num_val) : (num_train+2*num_val)], labels[(num_train+num_val):(num_train+2*num_val)]
  #X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.80,random_state=r)
  pipeline = Pipeline([('clf', LogisticRegression(random_state = r,class_weight="balanced"))])

  params = {'clf__C': [1.0,0.5,0.1], 'clf__max_iter':[500,1000]}


  rskf = StratifiedKFold(n_splits=6, random_state=r, shuffle=True)

  cv = GridSearchCV(pipeline, params, cv = rskf, scoring = 'accuracy')
  cv.fit(X_train, y_train)

  #print(f'Best accuracy -score: {cv.best_score_:.3f}\n')
  #print(f'Best parameter set: {cv.best_params_}\n')
  #print(f'Scores: {classification_report(y_train, cv.predict(X_train))}')

  preds_proba = cv.predict_proba(X_test)
  print(preds_proba)
  preds = cv.predict(X_test)
  #print(f'Scores: {classification_report(y_test, preds)}\n')
  #print(f'accuracy score: {accuracy_score(y_test, preds):.3f}')
  accuracy.append(accuracy_score(y_test, preds))
  F1_Score.append(f1_score(y_test, preds, average='weighted'))
  AUC.append(roc_auc_score(y_test,preds_proba,multi_class="ovo",average="weighted"))

print("Accuracy : ",np.mean(accuracy),"with std",np.std(accuracy))
print("f1_score : ",np.mean(F1_Score),"with std",np.std(F1_Score))
print("AUC : ",np.mean(AUC),"with std",np.std(AUC))

Random state :  0 

[[1.59937597e-01 5.73228150e-01 3.82885989e-02 4.98099195e-02
  1.71143223e-01 7.59251184e-03]
 [8.90187445e-03 9.90859054e-01 9.51266194e-05 9.12968046e-08
  1.43633388e-04 2.20626015e-07]
 [9.20267126e-01 4.90540777e-02 6.15744304e-03 3.90444268e-06
  7.42195861e-03 1.70954898e-02]
 ...
 [9.99178542e-01 4.39591130e-04 1.97663473e-04 5.46122121e-08
  1.60119843e-04 2.40290065e-05]
 [3.23164197e-02 7.35975502e-01 1.84345407e-01 3.06280728e-03
  3.85617393e-02 5.73812427e-03]
 [1.21289392e-01 6.95810443e-01 3.96711586e-03 2.16730243e-03
  1.76081366e-01 6.84379921e-04]]
Accuracy :  0.754 with std 0.0
f1_score :  0.7411147280702143 with std 0.0
AUC :  0.7118910024256633 with std 0.0


In [18]:
prob=cv.predict_proba(X_test)[:,1]
prob

array([6.04996663e-02, 5.11976678e-02, 9.62333802e-01, 3.60029621e-01,
       3.09094147e-01, 5.35698459e-02, 9.71597683e-01, 2.28595181e-01,
       7.53030180e-01, 3.72132104e-02, 6.28950329e-01, 9.52000889e-01,
       8.27293586e-01, 9.41255230e-01, 7.42909943e-04, 5.63497480e-04,
       2.28456282e-01, 1.43646156e-01, 5.22910866e-01, 8.29249694e-01,
       8.56235450e-01, 9.94581561e-01, 7.33236142e-01, 1.69439866e-03,
       6.34871614e-01, 1.96072982e-01, 6.28790637e-01, 6.12093124e-01,
       8.37934221e-02, 9.72960017e-01, 1.72605801e-01, 9.46420327e-01,
       6.10107827e-03, 1.94429491e-01, 3.38196905e-02, 1.02256655e-01,
       6.29695925e-01, 8.88590118e-01, 8.26984755e-01, 9.04378576e-01,
       4.62074010e-02, 4.71519484e-02, 9.99944098e-01, 4.00041977e-01,
       3.56564699e-01, 2.19026927e-01, 4.66967618e-02, 7.56502632e-02,
       2.64554232e-03, 6.76516781e-01, 1.57320010e-02, 6.69733027e-01,
       9.72776208e-01, 7.86973798e-01, 6.48524128e-01, 8.39465892e-01,
      

# LSTM

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        print("sentence:", sentence)
        #sentence=torch.from_numpy(sentence)
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
def sentence_process(sentence):
  comment = sentence.replace('\n', ' ').replace(':::::', '')
  while '\"' in comment :
    comment = comment.replace('\"', '')
  while "\'" in comment :
    comment = comment.replace("\'", '')
  return comment

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.LongTensor(idxs)
    #return torch.tensor(idxs, dtype=torch.long)
  
class ToxicCommentDataset:
    def __init__(self, n_words):
        super().__init__()
        self.n_words = n_words

    def process(self):
        tic = time.time()
        self.labels = []

        train_df = pd.read_csv('/content/drive/My Drive/NLP/train.csv')
        tic = time.time()

        #delete comments that belongs to several classes
        idx_to_del = []
        for row in train_df.itertuples():
          if sum(row[3::])> 1 :
            idx_to_del.append(row[0])
          
        train_df = train_df.drop(idx_to_del)

        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        model = GPT2Model.from_pretrained('gpt2')

        cpt = 0
        n_too_long = 0

        toxic = train_df[train_df['toxic'] == 0]
        obscene = toxic[toxic['obscene']==0]
        threat = obscene[obscene['threat']==0]
        insult = threat[threat['insult']==0]
        neutral = insult[insult['identity_hate']==0] #df with neutral comment

        toxic = train_df[train_df['toxic'] == 1]
        obscene = train_df[train_df['obscene'] == 1]
        threat = train_df[train_df['threat'] == 1]
        insult = train_df[train_df['insult'] == 1]
        identity_hate = train_df[train_df['identity_hate'] == 1]
        final_df = pd.concat([toxic, obscene, threat, insult, identity_hate, neutral]) #merge all the df together in the right order

        input_seq = []

        for row in final_df.itertuples():
      
          cpt = cpt + 1
          print("cpt:", cpt)

          if len(input_seq)== self.n_words:
            break
          
          comment = sentence_process(row[2])
         
          words = comment.split(' ')
          while '' in words :
            words.remove('')
          comment = ' '.join(words)

          if len(comment)>1024 : 
            print("Too long")
            n_too_long += 1
            continue

          #print("comment:", comment)
          if row[3] == 0 and row[4] == 0 and row[5] ==0 and row[6] ==0 and row[7] == 0 and row[8] == 0:
            self.labels.append(0)
          elif row[3] == 1 :
            self.labels.append(1)
          elif row[4] == 1 :
            self.labels.append(2)
          elif row[5] == 1 :
            self.labels.append(3)
          elif row[6] == 1 :
            self.labels.append(4)
          elif row[7] == 1 :
            self.labels.append(5)
          else: 
            self.labels.append(6)
      
          #Get embeddings for the sentence


          encoded_input = tokenizer(comment, return_tensors='pt')
          output = model(**encoded_input)
          outputseq = output.last_hidden_state[0]
          print("outputseq shape", outputseq.shape)
          
          input_seq.append(outputseq)
          

        input_seq = torch.tensor(np.array(input_seq))
        labels = torch.tensor(np.array(self.labels))
          
        return input_seq, labels

dataset = ToxicCommentDataset(n_words=5).process()
print(dataset)

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

cpt: 1
outputseq shape torch.Size([83, 768])
cpt: 2
outputseq shape torch.Size([19, 768])
cpt: 3
outputseq shape torch.Size([73, 768])
cpt: 4
outputseq shape torch.Size([86, 768])
cpt: 5
outputseq shape torch.Size([121, 768])
cpt: 6


RuntimeError: ignored

In [ ]:
import numpy as np
a = np.random.random((5000, 768*1024))

In [ ]:
from collections import Counter
#labels = ['cat', 'dog', 'mouse', 'elephant', 'pandas']
comment = 'John likes Mary'
words = comment.split(' ')
while '' in words :
  words.remove('')
comment = ' '.join(words)

le = preprocessing.LabelEncoder()
targets = le.fit_transform(words)
targets = np.array(targets)
targets

model = LSTMTagger(768, 1024, 661, 6)
#logit=model(targets)
#logit
tags = nltk.pos_tag(words)
tags


Counter({'NNP': 2, 'VBZ': 1})


['NNP', 'VBZ']